In [19]:
import numpy as np
import regex as re
import os
import math
import pandas as pd 
import sys
from openpyxl import load_workbook, Workbook
from openpyxl.utils.cell import get_column_letter

path_src = "/local/scratch/Figaro/figaro-code"
exp_names = ["mkl", "figaro_thick", "figaro_thin"]
exp_paths = {"figaro_thick": "dumps/figaro_thick", "mkl": "dumps/python/numpy", "figaro_thin": "dumps/figaro_thin"}
csv_name = "R.csv"

row_nums = [512]
column_nums =[2]
row_nums = [512, 1024, 2048, 4096]
column_nums = [1, 4, 16, 64, 256, 1024, 4096]
row_col_nums = []
np.set_printoptions(threshold=sys.maxsize, precision=14)
join_order = "FullJoin"

for row_num in row_nums:
    for col_num in column_nums:
        if (row_num > col_num):
            row_col_nums.append((row_num, col_num))

row_col_nums = row_col_nums[:21]

def arrange_up_triang(n: int):
    A  = np.zeros( (n, n))
    idx = 1
    for row_idx in range(n):
        for col_idx in range(row_idx, n):
            A[row_idx, col_idx] = idx
            idx += 1
    return A


def compare_rs(R: np.array, num_rows: int, num_cols: int):
    R_expX = arrange_up_triang(num_cols) * math.sqrt(num_rows)
    diff = R[:num_cols, :num_cols] - R_expX
    #print(R[:num_cols, :num_cols])
    #print(R_expX)
    #print(diff)
    rel_fro = np.linalg.norm(diff, ord='fro') / np.linalg.norm(R_expX, ord='fro')
    return rel_fro

exp_rel_errors = {}

for exp_name in exp_names:
    #Initialize paths for the current experiment 
    exp_path = exp_paths[exp_name]
    accur_path = os.path.join(path_src, exp_path)
    df_measurement = pd.DataFrame(columns=column_nums)
    dump_r_path = os.path.join(accur_path, csv_name)


    for db_idx, db_num in enumerate(range(1, len(row_col_nums))): 
        db_name = "DBCartesianProductAccuracy{}".format(db_num)
        path_csv = os.path.join(accur_path, db_name, join_order, csv_name)
        
        # Writng header files
        row_num = row_col_nums[db_idx][0]
        col_num = row_col_nums[db_idx][1]
        r_mat = np.genfromtxt(path_csv, delimiter=',')
        rel_error = compare_rs(r_mat, row_num, col_num)
        df_measurement.at[row_num, col_num] = rel_error
    
    df_measurement = df_measurement.astype(float)
    exp_rel_errors[exp_name] = df_measurement
    df_measurement.to_csv(exp_name+"comp.csv")
    print(exp_name)
    print(df_measurement.to_latex(float_format="%.2g"))
#print(exp_rel_errors)

mkl
\begin{tabular}{lrrrrrrr}
\toprule
{} &  1    &  4    &  16   &  64   &  256  &  1024 &  4096 \\
\midrule
512  &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN \\
1024 &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN \\
2048 &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN \\
4096 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN &   NaN \\
\bottomrule
\end{tabular}

figaro_thick
\begin{tabular}{lrrrrrrr}
\toprule
{} &  1    &  4    &  16   &  64   &  256  &  1024 &  4096 \\
\midrule
512  &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN \\
1024 &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN \\
2048 &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN \\
4096 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN &   NaN \\
\bottomrule
\end{tabular}

figaro_thin
\begin{tabular}{lrrrrrrr}
\toprule
{} &  1    &  4    &  16   &  64   &  256  &  1024 &  4096 \\
\midrule
512  &  0.00 &  0.00 &  0.00 &  0.00 &  0.00 &   NaN &   NaN \\
1024 &  0.00 &  0.00 & 